In [ ]:
# !pip install pdfquery
# !pip install pandas
# !pip install PyPDF2
# !pip install --upgrade pymupdf
# !pip install requests

In [1]:
import pandas as pd
import os
import fitz
import requests
import csv
import PyPDF2

In [2]:
os.chdir(r"C:\Users\SeanS\Documents\CS 6422\EvaDB PDF\orientation")

PyPDF2

In [3]:
with open('spring_2023_orientation_document.pdf', 'rb') as pdf_file:
    pdf_reader = PyPDF2.PdfReader(pdf_file)
    num_pages = len(pdf_reader.pages)
    
    extracted_text = ''
    for page_num in range(num_pages):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()

    with open('pypdf2.txt', 'w', encoding='utf-8') as txt_file:
        txt_file.write(extracted_text)

Fitz Only

In [4]:
doc = fitz.open("spring_2023_orientation_document.pdf") 
out = open("fitz_only.txt", "wb") 
for page in doc: 
    text = page.get_text().encode("utf8") # get plain text (is in UTF-8)
    out.write(text) 
    out.write(bytes((12,)))
out.close()

unstructuredIO only

In [11]:
url = 'https://api.unstructured.io/general/v0/general'

headers = {
    'accept': 'application/json',
    'unstructured-api-key': 'DTmC4RIyD6E3XMBJcucb3GOUMoKHWK',
}

data = {
    "strategy": "ocr_only",
    "ocr_languages": ["eng", "kor"]
}

file_path = "spring_2023_orientation_document.pdf"
file_data = {'files': open(file_path, 'rb')}

response = requests.post(url, headers=headers, data=data, files=file_data)

file_data['files'].close()

structured_data = response.json()

df = pd.json_normalize(structured_data)
print(df)

df.to_csv("unstructured_only.csv", index=False)

              type                        element_id  \
0            Title  083ea61c9b869089761a9e341155588c   
1            Title  661618e17731986974fb3da731b1b96b   
2            Title  7964f80b568e85e67060fac84a51786e   
3            Title  3bcd9e03e3eaa295031a24cf9838cbde   
4    NarrativeText  705eac3b0c92b0b88ed6179a97b0a8c7   
..             ...                               ...   
994          Title  c3d24dae34dc01c8baa600dd8d981014   
995          Title  97a022c9dfbdc8d6464bb9eb82c22ee4   
996       ListItem  e3b0c44298fc1c149afbf4c8996fb924   
997   EmailAddress  5d7a4c5738bdc8457d551fd74e74a950   
998  NarrativeText  fa47d600d89f84525001de8f0e5bbaf5   

                                                  text  \
0                                         Georgia Tech   
1                                 College of Computing   
2    Online Master of Science in Computer Science (...   
3               Spring 2023 OMSCS Orientation Document   
4    Welcome to the OMSCS program! Th

processing fitz_only.txt

In [9]:
with open("fitz_only.txt", "r", encoding='utf-8') as f:
    lines = f.readlines()

sentences = []
current_sentence = ''

for line in lines:
    stripped_line = line.strip()
    if not stripped_line:
        # skip empty lines
        continue  
    # If the line looks like a title (no ending punctuation and possibly short)
    if (len(stripped_line) < 60 and not stripped_line.endswith(('.', '!', '?'))):
        if current_sentence:
            sentences.append(current_sentence)
            current_sentence = ''
        # Treat title as its own 'sentence'
        sentences.append(stripped_line)  
        continue
    # If the line ends with a typical sentence-ending punctuation
    if stripped_line.endswith(('.', '!', '?')):
        current_sentence += ' ' + stripped_line
        sentences.append(current_sentence)
        current_sentence = ''
    else:
        current_sentence += ' ' + stripped_line

# For the case where the last piece of text doesn't end with a sentence-ending punctuation
if current_sentence:
    sentences.append(current_sentence)
    
with open('fitz_processed.txt', 'w', encoding='utf-8') as f:
    for sentence in sentences:
        f.write(sentence + '\n')

fitz + unstructuredIO

In [10]:
url = 'https://api.unstructured.io/general/v0/general'

headers = {
    'accept': 'application/json',
    'unstructured-api-key': 'DTmC4RIyD6E3XMBJcucb3GOUMoKHWK',
}

data = {
    "strategy": "ocr_only",
    "ocr_languages": ["eng", "kor"]
}

file_path = "fitz_processed.txt"
file_data = {'files': open(file_path, 'rb')}

response = requests.post(url, headers=headers, data=data, files=file_data)

file_data['files'].close()

structured_data = response.json()

df2 = pd.json_normalize(structured_data)
print(df2)

df.to_csv("fitz+unstructured.csv", index=False)

               type                        element_id  \
0             Title  71ff43c910647e81d39e44c90f1cda0b   
1             Title  492adf374353f048a3fcae9a48578d49   
2             Title  3bcd9e03e3eaa295031a24cf9838cbde   
3     NarrativeText  705eac3b0c92b0b88ed6179a97b0a8c7   
4     NarrativeText  1f1bc56e00d904291eb1716c7a6782fc   
...             ...                               ...   
1417          Title  5fea0f0a7249db6476df8c65e4d17696   
1418          Title  661618e17731986974fb3da731b1b96b   
1419          Title  c3d24dae34dc01c8baa600dd8d981014   
1420   EmailAddress  03861bef551800ced62de49b5e9b8a02   
1421  NarrativeText  fa47d600d89f84525001de8f0e5bbaf5   

                                                   text   metadata.filename  \
0     Online Master of Science in Computer Science (...  fitz_processed.txt   
1                                           Spring 2023  fitz_processed.txt   
2                Spring 2023 OMSCS Orientation Document  fitz_processed.txt   

Fitz can extract images :>

In [12]:
doc = fitz.open("spring_2023_orientation_document.pdf")
for page_index in range(len(doc)): # iterate over pdf pages
    page = doc[page_index] # get the page
    image_list = page.get_images()

    # print the number of images found on the page
    if image_list:
        print(f"Found {len(image_list)} images on page {page_index}")
    else:
        print("No images found on page", page_index)

    for image_index, img in enumerate(image_list, start=1): # enumerate the image list
        xref = img[0] # get the XREF of the image
        pix = fitz.Pixmap(doc, xref) # create a Pixmap

        if pix.n - pix.alpha > 3: # CMYK: convert to RGB first
            pix = fitz.Pixmap(fitz.csRGB, pix)

        pix.save("page_%s-image_%s.png" % (page_index, image_index)) # save the image as png
        pix = None
doc.close()

Found 1 images on page 0
No images found on page 1
No images found on page 2
No images found on page 3
No images found on page 4
No images found on page 5
No images found on page 6
No images found on page 7
No images found on page 8
No images found on page 9
No images found on page 10
No images found on page 11
No images found on page 12
No images found on page 13
No images found on page 14
No images found on page 15
No images found on page 16
No images found on page 17
No images found on page 18
No images found on page 19
No images found on page 20
No images found on page 21
No images found on page 22
No images found on page 23
No images found on page 24
No images found on page 25
No images found on page 26
No images found on page 27
No images found on page 28
No images found on page 29
No images found on page 30
No images found on page 31
No images found on page 32
No images found on page 33
No images found on page 34
No images found on page 35
No images found on page 36
No images fo